## Машинное обучение, ВМК МГУ

## Практическое задание 2

### Общая информация
Дата выдачи: 7 октября 2020

Максимальная оценка: 10 баллов + 1 бонусный балл

Мягкий дедлайн: 23:59MSK 20 октября

Жесткий дедлайн: 23:59MSK 27 октября.

### О задании

В этом задании вы:
- Познакомитесь с методом решения задачи регрессии на основе метода ближайших соседей.
- Реализуете алгоритм kNN для задачи регрессии.
- Изучите методы работы с категориальными и текстовыми переменными.


### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов.

Сдавать задание после указанного срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.


### Формат сдачи
Для сдачи задания переименуйте получившийся файл *.ipynb в соответствии со следующим форматом: homework-practice-02-Username.ipynb, где Username — ваша фамилия и имя на латинице именно в таком порядке (например, homework-practice-02-ivanov.ipynb).

Далее отправьте этот файл на anytask в соответсвующий раздел.

In [1]:
import numpy as np
import pandas as pd

Все эксперименты в этой лабораторной работе предлагается проводить на данных соревнования New York City Airbnb Open Data: https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data#AB_NYC_2019.csv

В данной задаче предлагается предсказать цену на съем квартиры в зависимости от её параметров.

In [2]:
data = pd.read_csv('AB_NYC_2019.csv')
data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [3]:
data.shape

(48895, 16)

In [4]:
# число значений у признаков
for col_name in data.columns:
    print(col_name, len(data[col_name].unique()))

id 48895
name 47906
host_id 37457
host_name 11453
neighbourhood_group 5
neighbourhood 221
latitude 19048
longitude 14718
room_type 3
price 674
minimum_nights 109
number_of_reviews 394
last_review 1765
reviews_per_month 938
calculated_host_listings_count 47
availability_365 366


In [5]:
data.isna().sum()

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [6]:
data[data["last_review"]!=data["last_review"]]["number_of_reviews"].mean()

0.0

Как видите, в данных есть пропуски. Не забудьте обработать их

In [7]:
data.fillna({"name": "missing", "host_name": "unknown", "last_review": "no", "reviews_per_month": 0}, inplace=True)
data.head(5)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,no,0.00,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


Разобъем данные на обучение и контроль.

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=['price']), data[['price']],
                                                    test_size=0.3, random_state=241)

## Часть 1: Алгоритм kNN в задаче регрессии

<b> 1.1 (1.5 балла) </b>
Реализуйте класс `KNNRegressor`, который используя метод k ближайших соседей решает задачу регрессии. Для решение данной задачи, необходимо найти $N_k$ - k соседей, и после использовать значения их целевых переменных для предсказания:
\begin{align}
y = \frac{1}{k}\sum_{n \in N_k}w_n y_n,
\end{align}

где $w_n$ - вес каждого соседа. 

При этом `KNNRegressor` может работать в 2 режимах:
 - $uniform$ - ближайшие соседи учитываются с одинаковыми весами.
 - $distance$ - вес ближайших соседей зависит от расстояния
 
**Сигнатуру методов, классов, функции при желании можно менять.**

In [9]:
from typing import Union, Callable, Iterable, Optional
from sklearn.neighbors import NearestNeighbors


class KNNRegressor:

    def __init__(self, n_neighbors: int, metric: Union[str, Callable],
                 mode: str = 'uniform', correct: float = 10e-5, **kwargs):
        """
        params:
            n_neighbors: number of neighbors
            metric: metric to use for distance computation
            mode: 'uniform' or 'distance'
            'uniform' - all points in each neighborhood are weighted equally
            'distance' - weight points by the inverse of their distance
            'correct' - additive value if distance will be zero
        """
        self.__nn = NearestNeighbors(n_neighbors=n_neighbors, metric=metric, algorithm="brute", **kwargs)
        self.__mode = mode
        self.__n_neighbors = n_neighbors
        self.__correct = correct

    def fit(self, X: np.array, y: np.array) -> None:
        """
            X: data
            y: labels
        """
        self.__y = y
        self.__nn.fit(X)

    def predict(self, X: np.array, n_neighbors: Optional[int] = None) -> np.array:
        """
            X: data
            n_neighbors: number of neighbors
        """
        if n_neighbors is None:
            n_neighbors = self.__n_neighbors
        matrix_pairwise, matrix_indexs = self.__nn.kneighbors(X, n_neighbors=n_neighbors, return_distance=True)
        y_values = np.apply_along_axis(lambda x: self.__y[x], arr=matrix_indexs, axis=1)

        if self.__mode == "distance":
            return np.average(y_values, weights=1/(y_values+self.__correct), axis=1)
        return y_values.mean(axis=1)

Проверим работает ли:

In [10]:
from sklearn.datasets import load_boston
X, y = load_boston(return_X_y=True)
model=KNNRegressor(2, metric="euclidean",mode="distance")
model.fit(X[:400], y[:400])
model.predict(X[450:]).round(1), y[450:]
# Вроде что-то похожее на правду

(array([18.6, 18.2, 21.2, 19.2, 16.7, 12.7, 18.6, 18.6, 12.6, 22.6, 12.6,
        22.2, 22.6, 22.2, 22.6,  9.8, 12.7,  9.8, 24.1, 22.6, 22.6, 16.9,
        22.6, 24.1, 18.2, 12.6, 13.7, 16.4, 19.2, 20.1, 22.6, 22.6, 22.6,
        22.6, 22.9, 22.6, 22.6, 22.6, 15.9, 18.2,  9.8, 10.9, 22.6, 21.4,
        20.5, 20.5, 27. , 27. , 23.9, 27. , 22.7, 22. , 28.1, 23.3, 25.5,
        25.9]),
 array([13.4, 15.2, 16.1, 17.8, 14.9, 14.1, 12.7, 13.5, 14.9, 20. , 16.4,
        17.7, 19.5, 20.2, 21.4, 19.9, 19. , 19.1, 19.1, 20.1, 19.9, 19.6,
        23.2, 29.8, 13.8, 13.3, 16.7, 12. , 14.6, 21.4, 23. , 23.7, 25. ,
        21.8, 20.6, 21.2, 19.1, 20.6, 15.2,  7. ,  8.1, 13.6, 20.1, 21.8,
        24.5, 23.1, 19.7, 18.3, 21.2, 17.5, 16.8, 22.4, 20.6, 23.9, 22. ,
        11.9]))

## Часть 2: Категориальные признаки

<b>2.1 (1 балл)</b>
Реализуйте три функции расстояния на категориальных признаках _(в этом задании только на них!)_ , которые обсуждались на [втором семинаре](https://github.com/mmp-mmro-team/mmp_mmro_fall_2020/blob/master/seminars/sem02_knn.pdf). Не забудьте, что KNNRegressor должен уметь работать с этими функциями расстояния. Как вариант, можно реализовать метрики как [user-defined distance](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html).

In [11]:
# https://stackoverflow.com/questions/10741346/numpy-most-efficient-frequency-counts-for-unique-values-in-an-array
# пришлось написать специальные функции, чтобы нормально добывать частоты значений значений из категорий

def freq_categorical(X: np.array, return_count: bool = True) -> list:
    tmp_list = []
    for i in range(X.shape[1]):
        y, count = np.unique(X[:, i], return_counts=True)
        if(not return_count):
            count = count/count.sum()
        tmp_list.append(dict(zip(y, count)))
    return tmp_list


def acc_frequency_categorical(X: np.array, return_count: bool = True) -> list:
    tmp_list = []
    for i in range(X.shape[1]):
        y, count = np.unique(X[:, i], return_counts=True)
        arg_sort = np.argsort(count)
        y = y[arg_sort]
        if(not return_count):
            count = count / count.sum()
        count = np.cumsum(count)
        tmp_list.append(dict(zip(y, count)))
    return tmp_list

In [12]:
def overlap(x, z, w: Union[str, np.array] = "uniform") -> float:
    if type(w) != str:
        return np.average(x != z, weights=w)
    return (x != z).mean()

# циклы - это боль, но чистым нампаем не получилось:(
def flattened_overlap(x, z, acc_freq: list, w: Union[str, np.array] = "uniform") -> float:
    tmp_freq = np.zeros_like(x, dtype=float)
    mask = (x == z)
    i = 0
    while(i < len(acc_freq)):
        if mask[i]:
            tmp_freq[i] = acc_freq[i].get(x[i], 0)
        i += 1

    if type(w) != str:
        return np.average((np.logical_not(mask) + mask*tmp_freq), weights=w)
    return (np.logical_not(mask) + mask*tmp_freq).mean()


def log_overlap(x, z, freq: list, correct: bool = True, w: Union[str, np.array] = "uniform") -> float:
    tmp_freq_x = np.zeros_like(x, dtype=float)
    tmp_freq_z = np.zeros_like(z, dtype=float)
    mask = (x != z)
    i = 0
    while(i < len(freq)):
        if mask[i]:
            tmp_freq_x[i] = freq[i].get(x[i], 0)
            tmp_freq_z[i] = freq[i].get(z[i], 0)
        i += 1
    if type(w) != str:
        return np.average((x != z)*np.log(1+tmp_freq_x)*np.log(1+tmp_freq_z), weights=w)
    return (mask*np.log(1+tmp_freq_x)*np.log(1+tmp_freq_z)).mean()

<b>2.2 (1 балл)</b> Найдите все категориальные признаки в данных. Подсчитайте для каждой из метрик качество на тестовой выборке `X_test` при числе соседей $k = 10$. Качество измеряйте с помощью RMSE.

Какая функция расстояния оказалась лучшей? Почему?

In [13]:
info=data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48895 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48895 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

**Категориальных признаков:4** (На самом деле можно по-разному выбрать категориальные данные, но я остановлюсь на таком  ↓)

In [14]:
from sklearn.preprocessing import OrdinalEncoder
cat = ["host_name", "neighbourhood_group", "neighbourhood", "room_type"]
encoder = OrdinalEncoder().fit(data[cat].to_numpy())
X_train_cat = encoder.transform(X_train[cat].to_numpy())
X_test_cat = encoder.transform(X_test[cat].to_numpy())

y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [15]:
freq = acc_frequency_categorical(X_train_cat)
acc_freq = acc_frequency_categorical(X_train_cat,return_count=False)

In [16]:
from sklearn.metrics import mean_squared_error

In [17]:
ind_test = np.random.randint(0, y_test.size, 100)

In [18]:
model = KNNRegressor(10, metric=flattened_overlap, mode="uniform", metric_params={"acc_freq": acc_freq})
model.fit(X_train_cat, y_train)
mean_squared_error(model.predict(X_test_cat[ind_test, :]), y_test[ind_test], squared=False)

140.7923396353651

In [19]:
model = KNNRegressor(10, metric=overlap, mode="uniform")
model.fit(X_train_cat, y_train)
mean_squared_error(model.predict(X_test_cat[ind_test, :]), y_test[ind_test], squared=False)

154.69013769468305

In [20]:
model = KNNRegressor(10, metric=log_overlap, mode="uniform", metric_params={"freq": freq})
model.fit(X_train_cat, y_train)
mean_squared_error(model.predict(X_test_cat[ind_test, :]), y_test[ind_test], squared=False)

131.70591786248633

<b>2.3 (1 балл) бонус</b> Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какого удалось достичь уровня качества?

In [199]:
models = {
    "flattened_overlap": KNNRegressor(10, metric=flattened_overlap, mode="uniform", metric_params={"acc_freq": acc_freq}),
    "overlap": KNNRegressor(10, metric=overlap, mode="uniform"),
    "log_overlab": KNNRegressor(10, metric=log_overlap, mode="uniform", metric_params={"freq": freq})
}
scores = {"flattened_overlap": [], "overlap": [], "log_overlab": []}
for m in models:
    models[m].fit(X_train_cat, y_train)

ind_test = np.random.randint(0, y_test.size, 80)

for i in range(10, 16):
    print(f"neighbors: {i}")
    for m in models:
        rmse = mean_squared_error(models[m].predict(X_test_cat[ind_test, :], n_neighbors=i),
                                  y_test[ind_test], squared=False)
        scores[m].append(round(rmse, 2))
        print(f"{m} RMSE: {round(rmse,2)}")

neighbors: 10
flattened_overlap RMSE: 99.11
overlap RMSE: 94.6
log_overlab RMSE: 122.98
neighbors: 11
flattened_overlap RMSE: 97.41
overlap RMSE: 93.57
log_overlab RMSE: 119.03
neighbors: 12
flattened_overlap RMSE: 95.18
overlap RMSE: 91.35
log_overlab RMSE: 115.05
neighbors: 13
flattened_overlap RMSE: 95.41
overlap RMSE: 91.76
log_overlab RMSE: 110.15
neighbors: 14
flattened_overlap RMSE: 93.18
overlap RMSE: 91.46
log_overlab RMSE: 108.58
neighbors: 15
flattened_overlap RMSE: 90.97
overlap RMSE: 91.34
log_overlab RMSE: 108.76


In [208]:
for s in scores:
    tmp = np.array(scores[s])
    print(f"{s}, optimal k: {10+tmp.argmin()} rmse {scores[s][tmp.argmin()]}")

flattened_overlap, optimal k: 15 rmse 90.97
overlap, optimal k: 15 rmse 91.34
log_overlab, optimal k: 14 rmse 108.58


<b>2.4 (2.5 балла)</b> Отойдем ненадолго от задачи регрессии и перейдём к задаче классификации: будем определять, являеться ли квартира дорогой $(target = 1)$ или дешевой $(target = 0)$. Будем считать дорогими квариры, цена которых выше среднего.

In [311]:
data['target'] = (data.price > data.price.mean()).astype(int)
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=['target']), data[['target']],
                                                    test_size=0.3, random_state=241)

In [312]:
from sklearn.preprocessing import OrdinalEncoder

cat = ["host_name", "neighbourhood_group", "neighbourhood", "room_type"]
encoder = OrdinalEncoder().fit(data[cat].to_numpy())
X_train_cat = encoder.transform(X_train[cat].to_numpy())
X_test_cat = encoder.transform(X_test[cat].to_numpy())

y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [323]:
y_train==1

array([[False],
       [ True],
       [False],
       ...,
       [False],
       [False],
       [False]])

Реализуйте счетчики, которые заменят категориальные признаки на вещественные.

А именно, для каждого категориального признака $f_j(x)$ необходимо сделать следующее:
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
\begin{align}
counts_j(c) = \sum_{i=1}^l [f_j(x_i) = c]
\end{align}
2. Число `successes` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
\begin{align}
successes_j(c) = \sum_{i=1}^l[f_j(x_i) = c][y_i = +1].
\end{align}
3. Сглаженное отношение двух предыдущих величин:
\begin{align}
p_j(c) = \frac{successes_j(c) + a}{counts_j(c) + b},
\end{align}

где $a$ и $b$ - априорные счетчики (например, a = 1, b = 2).

In [314]:
def val_count(a, uniq, count):
    tmp = np.where(uniq == a)[0]
    if(tmp.size == 0):
        return 0
    return count[tmp]


vec_val_count = np.vectorize(val_count, excluded=[1, 2])

In [357]:
def counters(x, indexs: Optional[np.array] = None,
             uniq: Optional[np.array] = None, count: Optional[np.array] = None) -> np.array:
    """
    params:
        x: value on categorical feature for N objects
        indexs: where search unique elements of x(boolean or int array)
        uniq: already uniq 
        count: already uniq
    returns: vector of length N
    """
    if(uniq is None and uniq is None):
        uniq, count = np.unique(x[indexs], return_counts=True)
    return vec_val_count(x, uniq, count)

In [358]:
A = np.array([1, 1, 1, 3, 3, 4])
counters(A)

array([3, 3, 3, 2, 2, 1])

Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `successes` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанный по всей обучающей выборке. Реализуйте и такой вариант. Достаточно взять $n = 3$.

In [395]:
from sklearn.model_selection import KFold


def fold_counters(x, n: int = 3, indexs: Optional[np.array] = None):
    """
    params:
        x: value on categorical feature for N objects
        indexs: where search unique elements of x(boolean array)
    returns: vector of length N
    """

    kf = KFold(n).split(x)
    ans = np.zeros_like(x, dtype=int)
    mask = None
    for other, arr in kf:
        if indexs is not None:
            if indexs.dtype == int:
                mask = indexs[indexs < other.size]
            elif indexs.dtype == bool:
                mask = indexs[other].ravel()
        uniq, count = np.unique(x[other[mask]], return_counts=True)
        ans[arr] = vec_val_count(x[arr], uniq, count)
    return ans

In [396]:
A = np.array([1, 1, 1, 3, 3, 4])
fold_counters(A, indexs=(A == 1))

array([1, 1, 2, 0, 0, 0])

In [ ]:
a=1
b=2
X_train_count = np.apply_along_axis(fold_counters, arr=X_train_cat, axis=0)
X_train_suc = np.apply_along_axis(fold_counters, arr=X_train_cat, axis=0, indexs=(y_train == 1))
X_train_p = (X_train_suc+a)/(X_train_count+b)

Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

In [ ]:
# Ваш код здесь

<b>2.5 (1 балл)</b> Вернемся к задаче регрессии. Утверждается, что для задачи регрессии можно также сделать преобразование категориальных признаков в действительные числа. Для этого достаточно для каждого значения признака $f_j$ вычислить:
\begin{align}
p_j(c) = g(T_i | f_j(x_i) = c),
\end{align}

где $T_i$ - значения целевой переменной объекта $x_i$. Функция $g$ - среднее (mean) или среднеквадратичное отклонение (std).

Закодируйте категориальные признаки обоими способами и найдите значение RMSE. Используйте евклидову метрику для поиска ближайших соседей. Для какой функции $g$ значение RMSE лучше? Почему?

In [ ]:
# Ваш код здесь

## Часть 3: Текстовые признаки

<b>3.1 (2 балла)</b> Перейдем от категориальным признаков к текстовым. Рассмотрим 2 способа преобразования текста в действительные числа:
- Мешок слов (Bag of Words)
- TF-IDF

[Здесь](https://scikit-learn.org/stable/modules/feature_extraction.html) вы можете прочитать про их применение в Питоне.

Сравните оба способа на задаче регресси. Какую лучше метрику использовать: евклидову или косинусную меру? Постройте графики зависимости качества решения задачи от способа преобразования, метрики и количества соседей. Мера качества - RMSE.

Объясните полученные результаты.

Перед преобразованием не забудьте уменьшить размер словаря. Например, это можно сделать за счет приведения всех слов к одному регистру и удаления [стопслов](https://en.wikipedia.org/wiki/Stop_words) (артиклей, предлогов, союзов). Можно использовать использовать, например, библиотеку nltk.

In [ ]:
# Ваш код здесь

<b>3.2 (1 балл)</b> Используя все доступные признаки, решите задачу регрессии. Для категориальных и текстовых признаков выберите лучшие преобразования (из сделанных раннее заданий). Повлияло ли добавление количественного признака на метрику качества?

In [7]:
# Ваш код здесь

## Часть 4: Выводы


In [6]:
# Ваши выводы здесь (ノ°∀°)ノ⌒･*:.｡. .｡.:*･゜ﾟ･*☆